# Сессии. Авторизация. Аутентификация. WTF?!

# Зачем нужны замки на дверях?

![](img/70.jpg)

# Что можно делать, когда кто-то приходит?

# Понятия

![](img/71.jpg)

* Идентификация
* Аутентификация
* Авторизация

# Идентификация

### Мы пытаемся понять, кто это

**Идентификация** -  установление тождественности неизвестного объекта известному на основании совпадения признаков, опознание:
* известный нам объект, данные которые у нас есть;
* неизвестный нам объект.

# Аутентификация vs авторизация

# Аутентификация

### Мы пытаемся понять, за того ли себя выдает объект

**Аутентификация** - проверка подлинности объекта на основании сравнения признаков:
* проверка подлинности файла (на основании контрольной суммы);
* проверка подлинности пользователя (на основании совпадения логина и пароля с теми, которыми мы располагаем).

# Авторизация

### Мы ЗНАЕМ, КТО ПРИШЕЛ. Мы хотим определить, может ли этот кто-то сделать то, что он хочет

**Авторизация** - предоставление лицу или группе лиц прав на выполнение определённых действий.

![](img/72.png)

# В чем проблема HTTP?

### HTTP не сохраняет состояние между запросами. Нет возможности понять, от кого пришел запрос - от человека, который 5 минут назад зашел или от мошенника.

# Почему?

1. Запросы к серверу приходят хаотично - нельзя открыть соединение на все время, пока вы залогинены.
2. В запросе нет никакой информации о том, от кого он пришел.

Из-за этого сервер не может различить запросы.

# Выход? 

![](img/73.gif)

**Cookie** (куки) - кусок данных (4 Кбайт), который **хранится на стороне клиента** и **добавляется к каждому запросу**. Благодаря этому реализуется **сохранение промежуточного состояния**.

# Минусы

* ограниченный размер;
* ограниченное количество;
* КУКИ МОГУТ УКРАСТЬ С КОМПЬЮТЕРА И ОНИ БУДУТ РАБОТАТЬ НА ЧУЖОМ!!!

![](img/74.jpg)

# Делаем свою процедуру входа и используем сессии

![](img/75.jpg)

# Что ОБЯЗАТЕЛЬНО нужно сделать в начале?

1. Придумать, как и где будем хранить информацию о пользователях, которые могут входить на сайт.
2. Придумать, как мы будем делать аутентификацию.

# Как храним пользователей?

### Ruby models

```bash
~$ rails g model User email:string
```

Получаем

`app/models/user.rb`
```ruby
class User < ApplicationRecord
end
```

`db/migrate/<...>_create_users.rb`
```ruby
class CreateUsers < ActiveRecord::Migration[5.1]
  def change
    create_table :users do |t|
      t.string :email

      t.timestamps
    end
  end
end
```

# Сразу добавили безопасный пароль. Также сразу пользуемся функционалом рельсов для сравнения и проверки паролей

### Раскомментируем строчку в Gemfile

```ruby
# Use ActiveModel has_secure_password
gem 'bcrypt', '~> 3.1.7'
```

### Обновляем модель пользователя

```ruby
class User < ApplicationRecord
  has_secure_password
end
```

### При создании пользователя теперь доступны методы `password` и `password_confirmation`.

### ОБЯЗАТЕЛЬНО добавить поле password_digest (имя ВАЖНО!)

### Также для пользователя доступен метод `authenticate`

# Как логиним пользователей?

```ruby
class AuthExampleController < ApplicationController
  def we_need_check_auth_here
    # ...
    
    if user && user.authenticate(password)
      # Делаем что-то.
    else
      # Ругаемся
    end
  end
end
```

# Новый вариант

```ruby
class AuthExampleController < ApplicationController
  def we_need_check_auth_here
    # ...
    
    if user&.authenticate(password)
      # Делаем что-то.
    else
      # Ругаемся
    end
  end
end
```

# Храним юзера в сессии между запросами

Наивная реализация

```ruby
session[:remember_token] = user.id
```

Лучше

```ruby
session[:remember_token] = # Какая-то длинная случайная строка. Причем ее же нужно записать в ПОЛЬЗОВАТЕЛЯ
```